In [50]:
pip install pandas transformers


In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
import pandas as pd

def count_words(text):
    words = text.split()
    return len(words)

def main(input_csv_file, output_csv_file):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(input_csv_file)

    # Calculate the number of words in the comment section and create a boolean mask
    df['word_count'] = df['comment'].apply(count_words)
    mask = df['word_count'] <= 100

    # Filter the DataFrame based on the mask
    df_ytb = df[mask]

    # Drop the 'word_count' column as it's no longer needed
    df_ytb = df_ytb.drop(columns=['word_count'])

    # Reset index to rearrange numbering
    df_ytb = df_ytb.reset_index(drop=True)

    # Write the filtered DataFrame to a new CSV file
    df_ytb.to_csv(output_csv_file, index=False)

    return df_ytb

if __name__ == "__main__":
    input_csv_file = "/content/drive/MyDrive/CSV FILES/ytb_How Fast Was.csv"
    output_csv_file = "output.csv"

    df_ytb = main(input_csv_file, output_csv_file)
    print(df_ytb)

    Unnamed: 0                                            comment  \
0            0  A fabulous movie ! James Cain&#39;s story does...   
1            1  This is when I realise that the Neff character...   
2            2                           This movie is soooo good   
3            3  The smell of honeysuckle all up &amp; down tha...   
4            4  Look at how close she stays to him on the walk...   
..         ...                                                ...   
92          94  One of my favorite movies ever. Barbara Stanwy...   
93          95                              i love you too Keyes    
94          96  &quot;I wonder if you wonder&quot;.  A Raymond...   
95          97  This dialogue blew my mind the first time I he...   
96          98                   One of the best movies ever made   

             username  
0           DM McHugh  
1         Daniel Eyre  
2               Sofia  
3         snapfinger1  
4   chuck fitzpatrick  
..                ...  
92 

In [53]:
import csv
import re
import string
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

#df_ytb = pd.read_csv("/content/drive/MyDrive/CSV FILES/ytb_Kingdom of Us.csv")

# Remove column name 'Unnamed: 0'
dfc = df_ytb.drop(['Unnamed: 0','username'], axis=1)
# dfc = df_ytb.drop(index=11)
dfc = dfc.reset_index(drop=True)

#list_of_sentences = dfc['comment'].tolist()
#dfc
def clean_html(text):
    clean=re.compile('<.*?>')
    cleantext=re.sub(clean,'',text)
    return cleantext

# first round of cleaning
def clean_text1(text):
    text=text.lower()
    text=re.sub('','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\w*\d\w*','',text)
    return text

# second round of cleaning
def clean_text2(text):
    text=re.sub('[''"",,,]','',text)
    text=re.sub('\n','',text)
    return text

cleaned_html=lambda x:clean_html(x)
cleaned1=lambda x:clean_text1(x)
cleaned2=lambda x:clean_text2(x)

dfc['review'] = dfc['comment']
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned_html))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned1))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned2))
print(dfc.shape)
dfc.head(10)

first10comments = dfc.review.values

sentences = first10comments
list_of_sentences = dfc['comment'].tolist()


(97, 2)


In [54]:
tokenizer = AutoTokenizer.from_pretrained("aiknowyou/it-emotion-analyzer")
model = AutoModelForSequenceClassification.from_pretrained("aiknowyou/it-emotion-analyzer")

emotion_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
emotion_analysis(list_of_sentences)
results = emotion_analysis(list_of_sentences)
results

[{'label': '1', 'score': 0.46346592903137207},
 {'label': '1', 'score': 0.7900624871253967},
 {'label': '1', 'score': 0.762326717376709},
 {'label': '2', 'score': 0.5062981247901917},
 {'label': '1', 'score': 0.48718348145484924},
 {'label': '2', 'score': 0.949026346206665},
 {'label': '2', 'score': 0.8056222200393677},
 {'label': '2', 'score': 0.3622901439666748},
 {'label': '2', 'score': 0.5718850493431091},
 {'label': '1', 'score': 0.6871674656867981},
 {'label': '1', 'score': 0.676946759223938},
 {'label': '1', 'score': 0.5919030904769897},
 {'label': '2', 'score': 0.7371470332145691},
 {'label': '1', 'score': 0.8340431451797485},
 {'label': '1', 'score': 0.6736477613449097},
 {'label': '1', 'score': 0.7160174250602722},
 {'label': '1', 'score': 0.7544561624526978},
 {'label': '2', 'score': 0.6772704720497131},
 {'label': '1', 'score': 0.7097103595733643},
 {'label': '1', 'score': 0.7684134840965271},
 {'label': '1', 'score': 0.5310795903205872},
 {'label': '0', 'score': 0.57540786

In [55]:
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
# Map numerical labels to emotions in the results
dfc['Emotion'] = [emotion_mapping.get(result['label'], result['label']) for result in results]
dfc['Score'] = [result['score'] for result in results]

final =dfc[['comment', 'Emotion', 'Score']]
final


,comment,Emotion,Score
0,A fabulous movie ! James Cain&#39;s story does...,1,0.463466
1,This is when I realise that the Neff character...,1,0.790062
2,This movie is soooo good,1,0.762327
3,The smell of honeysuckle all up &amp; down tha...,2,0.506298
4,Look at how close she stays to him on the walk...,1,0.487183
...,...,...,...
92,One of my favorite movies ever. Barbara Stanwy...,2,0.518163
93,i love you too Keyes,2,0.961662
94,&quot;I wonder if you wonder&quot;. A Raymond...,2,0.614961
95,This dialogue blew my mind the first time I he...,2,0.504683


In [56]:
# Create a new column 'mapped_emotion' using emotion_mapping
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')

final['Mapped_Emotion'] = final['Emotion'].map(emotion_mapping)

# Display the updated 'final' DataFrame with the new column
print(final[['Emotion', 'Mapped_Emotion']])

    Emotion Mapped_Emotion
0         1            joy
1         1            joy
2         1            joy
3         2           love
4         1            joy
..      ...            ...
92        2           love
93        2           love
94        2           love
95        2           love
96        1            joy

[97 rows x 2 columns]


<ipython-input-56-a9bc13b09b5c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')


In [57]:
# Add sentiment analysis results to the DataFrame
# Save the DataFrame with sentiment results to a new CSV file
output_csv_path = "/content/drive/MyDrive/ytb_How Fast Was_with_sentiments.csv"
final.to_csv(output_csv_path, index=False)

# Display the first few rows of the DataFrame with sentiment results
final

,comment,Emotion,Score,Mapped_Emotion
0,A fabulous movie ! James Cain&#39;s story does...,1,0.463466,joy
1,This is when I realise that the Neff character...,1,0.790062,joy
2,This movie is soooo good,1,0.762327,joy
3,The smell of honeysuckle all up &amp; down tha...,2,0.506298,love
4,Look at how close she stays to him on the walk...,1,0.487183,joy
...,...,...,...,...
92,One of my favorite movies ever. Barbara Stanwy...,2,0.518163,love
93,i love you too Keyes,2,0.961662,love
94,&quot;I wonder if you wonder&quot;. A Raymond...,2,0.614961,love
95,This dialogue blew my mind the first time I he...,2,0.504683,love
